In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from collections import defaultdict

def calculate_constant(mass, gravity, C_Drag, diameter, rho):
    A = np.pi * (diameter/2)**2
    a = np.array([0, -gravity, 0])  # Gravity acts downward
    b_const = -(C_Drag * rho * A) / (2 * mass)
    return a, np.array([b_const]*3)

def ode_system(t, state, a, b):
    x, y, z, Vx, Vy, Vz = state
    speed = np.sqrt(Vx**2 + Vy**2 + Vz**2)
    return [
        Vx,
        Vy,
        Vz,
        a[0] + b[0] * Vx * speed,
        a[1] + b[1] * Vy * speed,
        a[2] + b[2] * Vz * speed
    ]

def check_collision(state, backpost_x, y_min, y_max):
    x, y = state[0], state[1]
    return backpost_x <= x <= backpost_x + 0.1 and y_min <= y <= y_max

def check_hit(Y_target, Y_tol, Z_target, Z_tol, Y_pos, X_pos, Z_pos, Prev_X_hit):
    mask = np.logical_and(
        np.isclose(Y_pos, Y_target, atol=Y_tol),
        np.isclose(Z_pos, Z_target, atol=Z_tol)
    )
    hit_indices = np.where(mask)[0]
    if hit_indices.size > 0:
        return X_pos[hit_indices[0]], X_pos[hit_indices[0]]
    return Prev_X_hit, Prev_X_hit

def simulate(initial_state, backpost_x, y_min, y_max, a, b, 
             X_target, Y_target, Z_target, tolerances, max_bounces=2):
    bounce_count = 0
    Score = 0
    time = 0
    X_hit = 0
    Prev_X_hit = 0
    max_time = 2 * X_target / (0.5 * np.linalg.norm(initial_state[3:6])) if np.linalg.norm(initial_state[3:6]) > 0 else 10

    while bounce_count <= max_bounces and time < max_time:
        t_span = (time, time + 0.001)
        sol = solve_ivp(ode_system, t_span, initial_state, args=(a, b), 
                       dense_output=True, vectorized=True)
        
        if not sol.success:
            break
            
        t_eval = np.linspace(*t_span, 1000)
        state = sol.sol(t_eval)
        
        # Check for collisions in entire trajectory
        collision_points = [check_collision(state[:,i], backpost_x, y_min, y_max) 
                           for i in range(state.shape[1])]
        if any(collision_points):
            collision_idx = collision_points.index(True)
            initial_state = state[:, collision_idx].copy()
            initial_state[3] *= -0.7  # Bounce damping
            initial_state[4] *= 0.5
            bounce_count += 1
            continue
            
        X_hit, Prev_X_hit = check_hit(Y_target, tolerances[1], Z_target, tolerances[2],
                                     state[1], state[0], state[2], Prev_X_hit)
        
        if np.isclose(X_hit, X_target, atol=tolerances[0]):
            Score = 1
            break
            
        if state[1, -1] < 0:  # Ground collision
            break
            
        initial_state = state[:, -1]
        time += 0.001

    return Score

def calculate_BestShot(SimData):
    param_scores = defaultdict(int)
    for V, Theta, phi, score in SimData:
        if score == 1:
            # Count neighbors with similar parameters that also succeeded
            neighbors = sum(1 for v,t,p,s in SimData 
                          if abs(v-V) <= 1 and abs(t-Theta) <= 5 and abs(p-phi) <= 5 and s == 1)
            param_scores[(V,Theta,phi)] = neighbors
    
    if not param_scores:
        return None
    
    best_params = max(param_scores.items(), key=lambda x: x[1])
    return best_params[0]

# Main Program
if __name__ == "__main__":
    # Constants
    MASS = 0.600
    DIAMETER = 0.240
    C_DRAG = 0.54
    RHO = 1.204
    GRAVITY = 9.81
    TOLERANCES = (0.11, 0.02, 0.11)  # X, Y, Z
    
    a, b = calculate_constant(MASS, GRAVITY, C_DRAG, DIAMETER, RHO)
    
    # Parameter space
    R_range = np.arange(1, 10, 1)
    V_range = np.arange(5, 15, 0.5)  # Reduced resolution for speed
    Theta_range = np.arange(30, 60, 2)  # More realistic angles
    Phi_range = np.arange(-45, 45, 5)    # Azimuth angles
    
    SimData = []
    
    for R in R_range:
        print(f"Processing R = {R}m...")
        for V in V_range:
            for Theta in Theta_range:
                for phi in Phi_range:
                    # Correct spherical coordinates conversion
                    Vx = V * np.cos(np.radians(Theta)) * np.sin(np.radians(phi))
                    Vy = V * np.sin(np.radians(Theta))
                    Vz = V * np.cos(np.radians(Theta)) * np.cos(np.radians(phi))
                    
                    initial_state = [0, 0.02, 0, Vx, Vy, Vz]
                    
                    score = simulate(
                        initial_state=initial_state,
                        backpost_x=R + 0.24,
                        y_min=1.9,
                        y_max=2.9,
                        a=a,
                        b=b,
                        X_target=R,
                        Y_target=2.43,
                        Z_target=0,
                        tolerances=TOLERANCES
                    )
                    
                    SimData.append((V, Theta, phi, score))
    
    optimal = calculate_BestShot(SimData)
    if optimal:
        print(f"\nOptimal parameters (Velocity, Theta, Phi): {optimal}")
        print("This configuration remains effective with ±1m/s velocity, ±5° angle variations")
    else:
        print("No successful configurations found")

Processing R = 1m...


KeyboardInterrupt: 